In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from pandas import DataFrame as DF
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import os
import ast
from math import log2, ceil
from util import get_notes_from
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#  Read tracks dataset
df = pd.read_csv("./fma_metadata/tracks.csv", header=[0,1,2])
df = df.filter(items=[
    ("Unnamed: 0_level_0","Unnamed: 0_level_1","track_id"),
    ("track", "genres","Unnamed: 41_level_2")
])
df.columns = ["id","genres"]
# Read genre dataset
df_genre = pd.read_csv("./fma_metadata/genres.csv")
df_genre = df_genre.filter(items=[
    "genre_id",
    "title"
])
df_genre.columns = ["id", "title"]

# Get required tracks
avail_tracks = [i for i in os.walk("fma_small_wav")]
avail_tracks = [int(i[:-4]) for j in range(1,157) for i in avail_tracks[j][2]]
df = df[df.id.isin(avail_tracks)]

# Get required genres
avail_genres = set([i for j in df.genres for i in ast.literal_eval(j)])
df_genre = df_genre[df_genre.id.isin(avail_genres)]

# Save first genre
df = df.apply(lambda x : [x.id, ast.literal_eval(x.genres)[0]], axis = 1)

In [17]:
def one_hot_vector(x):
    base_vector = [0.0 for i in range(0,113)]
    base_vector[x] = 1.0
    return base_vector

# Shuffle dataset
df = shuffle(df, random_state=1)
# Split dataset
labels = [one_hot_vector(list(df_genre.id).index(i)) for i in list(df.genres)]
train_x, test_x, train_y, test_y = train_test_split(list(df.id), labels, test_size=0.2, random_state=400)

In [9]:
# Define hyperparameters
learning_rate = 0.2
training_epochs = 1000
cost_history = np.empty(shape=[1], dtype=float)
n_class = df_genre.shape[0]
n_bits = 2 ** ceil(log2(n_class))
model_path = os.path.abspath("./model/")
n_input = 30
print(
    "==== Hyperparameters ====\n",
    "Learning Rate:   " + str(learning_rate) + "\n",
    "Epochs:          " + str(training_epochs) + "\n",
    "Cost History:    " + str(cost_history) + "\n",
    "Number of class: " + str(n_class) + "\n",
    "Model path:      " + str(model_path) 
)

# Number of layers & units per layer
n_hidden_1 = 30
n_hidden_2 = 30
n_hidden_3 = 30
n_hidden_4 = 30

x = tf.placeholder(tf.float32, [n_input, 1])
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32, [n_input, 1])

def perceptron(x, weights, biases):
    
    layer_1 = tf.add(tf.matmul(weigths["h1"], x), biases["h1"])
    layer_1 = tf.nn.sigmoid(layer_1)
    
    layer_2 = tf.add(tf.matmul(weigths["h2"], layer_1), biases["h2"])
    layer_2 = tf.nn.sigmoid(layer_2)
    
    layer_3 = tf.add(tf.matmul(weigths["h3"], layer_2), biases["h3"])
    layer_3 = tf.nn.sigmoid(layer_3)
    
    layer_4 = tf.add(tf.matmul(weigths["h4"], layer_3), biases["h4"])
    layer_4 = tf.nn.relu(layer_4)
    
    out_layer = tf.matmul(weigths["out"], layer_4) + biases["out"]
    return out_layer

weigths = {
    "h1": tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
    "h2": tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
    "h3": tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3])),
    "h4": tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4])),
    "out": tf.Variable(tf.truncated_normal([n_hidden_4, n_class])),
}

biases = {
    "h1": tf.Variable(tf.truncated_normal([n_hidden_1])),
    "h2": tf.Variable(tf.truncated_normal([n_hidden_2])),
    "h3": tf.Variable(tf.truncated_normal([n_hidden_3])),
    "h4": tf.Variable(tf.truncated_normal([n_hidden_4])),
    "out": tf.Variable(tf.truncated_normal([n_class])),
}

init = tf.global_variables_initializer()

saver = tf.train.Saver()

y = perceptron(x, weigths, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(init)

mse_history = []
accuracy_history = []

for epoch in range(training_epochs):
    sess.run(training_step, feed_dict={x: train_x, y: train_y})
    cost_history = np.append(cost_history, cost)
    correct_prediction = 0 # change me
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    pred_y = sess.run(y, feed_dict={x: test_x})
    mse = tf.reduce_mean(tf.square(pred_y - test_y))
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    accuracy = (sess.run(accuracy, feed_dict={x: train_x, y_: train_y}))
    accuracy_history.append(accuracy)
    print('epoch: ', epoch, ' -  cost: ', cost, ' - MSE: ', mse_, ' - Acc: ', accuracy)

save_path = saver.save(sess, model_path)
print('Model saved in file ', save_path)

plt.plot(mse_history, 'r')
plt.show()
plt.plot(accuracy_history)
plt.show()



==== Hyperparameters ====
 Learning Rate:   0.2
 Epochs:          1000
 Cost History:    [2.17e-322]
 Number of class: 113
 Number of bits:  128
 Model path:      C:\MUSIC_PROJECT\model


ValueError: Dimensions must be equal, but are 113 and 30 for 'MatMul_20' (op: 'MatMul') with input shapes: [30,113], [30,30].